In [63]:
"""
Acknowledgement
---------------
1. set your browser download location to your desire location because all files will be downloaded to the download location setted in your browser.
2. create these files -> files_info.json, bot.log, .secret

Thanks -> pythonjar, MariyaSha and some other stackoverflow members...

"""


'\nAcknowledgement\n---------------\n1. set your browser download location to your desire location because all files will be downloaded to the download location setted in your browser.\n2. create these files -> files_info.json, bot.log, .secret\n\nThanks -> pythonjar, MariyaSha and some other stackoverflow members...\n\n'

In [1]:
import os
import math
import time
import glob
import json
import string
import platform
import unicodedata
from selenium import webdriver
from json.decoder import JSONDecodeError
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [2]:
encoding = "utf-8"
target_file_type = "pdf"
log_file_location = "bot.log"

# This file keeps track of every downloaded file
tracker_file_location = "files_info.json"

explicit_wait_time = 10
network_failure_timeout = 600

# this value will ensure how many downloads will happen concurrently
download_thread_count = 3

"""
 targets identifiers -> this identifiers will change continuously so update it according to your needs 

"""
download_button_cssSelector = "a[href*='https://www.facebook.com/download/']"
fileOption_xpath = "//div[@aria-label='File options']"  # by default finds 15 per-scroll


# For obvious reason 😉
with open(".secret", encoding=encoding) as f:
    """
    0 -> username,
    1 -> password,
    2 -> download directory (for checking)
    3 -> chromewebdriver location (windows)

    """
    secrets = [secret.strip() for secret in f.readlines()]

# Ignore all alerts from the webpage
options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications": 2}
options.add_experimental_option("prefs", prefs)

# Tested on Windows & Linux
if platform.system() == "Windows":
    service = Service(secrets[3])
    driver = webdriver.Chrome(service=service, options=options)
elif platform.system() == "Linux":
    driver = webdriver.Chrome("chromedriver", options=options)


In [3]:
wait = WebDriverWait(
    driver, explicit_wait_time
)  # to wait until the element is ready -> Explicit Waits

# open the webpage
driver.get("http://www.facebook.com")

# target credentials
username = wait.until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']"))
)
password = wait.until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']"))
)

# enter username and password
username.clear()
username.send_keys(secrets[0])
password.clear()
password.send_keys(secrets[1])

# target thesrc login button and click it
button = (
    WebDriverWait(driver, 2)
    .until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']")))
    .click()
)

""" It should be logged in! -> if you have 2fa then you have to authorize it manually and then run the remaining cells manually """


' It should be logged in! -> if you have 2fa then you have to authorize it manually and then run the remaining cells manually '

In [4]:
#########################################################
# sleep is important because if we scrape too fast then #
# facebook will detect the bot and block this account   #
#########################################################
def waitNSeconds(sleep_time=1):
    time.sleep(sleep_time)


def compareString(s1, s2):
    # To avoid comparison between different unicode char
    s1 = unicodedata.normalize("NFC", s1)
    s2 = unicodedata.normalize("NFC", s2)

    # Removing whitespace in the string before comparing
    # because when file saves in the machine it seems to add whitepsaces after '-'
    # remove = string.punctuation + string.whitespace
    remove = string.whitespace
    mapping = {ord(c): None for c in remove}

    return s1.translate(mapping) == s2.translate(mapping)


def getExistingFilesInfo():
    with open(tracker_file_location, "r", encoding=encoding) as f:
        try:
            files_info = json.load(f)["files"]
        except JSONDecodeError:
            files_info = []

    # already downloaded file's name in download directory
    downloaded_files = sorted(
        [os.path.basename(f) for f in glob.glob(secrets[2] + "/*." + target_file_type)]
    )

    # files that are registered in files_info.json
    tracked_files = sorted(
        [[info["name"], info["uploaded_date"]] for info in files_info]
    )

    return [downloaded_files, tracked_files]


def appendFilesInfo(res):
    with open(tracker_file_location, "r+", encoding=encoding) as f:
        try:
            data = json.load(f)
            data["files"].append(res)

            # Sets file's current position at offset.
            f.seek(0)
            json.dump(data, f, indent=4, ensure_ascii=False)

            data.clear()
        except JSONDecodeError:
            # If the file is empty initially
            data = {"files": [res]}
            json.dump(data, f, indent=4, ensure_ascii=False)


def updateLog(text):
    print(text)  # Unnecessary but why not :3
    with open(log_file_location, "a", encoding=encoding) as f:
        f.write(text + "\n")


def binarySearch(item, itemList, multipleCheck=False):
    left = 0
    right = len(itemList) - 1

    while left <= right:
        mid = math.floor(left + (right - left) / 2)

        if multipleCheck:
            # Checking both file name and file uploaded date
            # if each == itemList[mid][idx] for idx, each in enumerate(item):
            if compareString(itemList[mid][0], item[0]) and compareString(
                itemList[mid][1], item[1]
            ):
                return mid
            elif itemList[mid][0] > item[0]:
                right = mid - 1
            else:
                left = mid + 1
        else:
            if compareString(itemList[mid], item):
                return mid
            elif itemList[mid] > item:
                right = mid - 1
            else:
                left = mid + 1

    return -1


def isDownloaded(fileName, uploadDate, downloaded_files, tracked_files):
    """
    if either tracker file or download directory is empty then there was no previous
    attempt to download these files in this case all files should be downloaded

    """
    if not downloaded_files or not tracked_files:
        return False

    """ If the file exist both in tracker file(files_info.json) & in the download location then the file has been downloaded """
    downloadedFileIndex = binarySearch(fileName, downloaded_files)
    trackedFileIndex = binarySearch((fileName, uploadDate), tracked_files, True)

    """ if the file is not present in the download directory then it was not downloaded """
    # if not downloadedFileIndex: # because index can be 0
    if downloadedFileIndex == -1:
        return False

    """
    if the file is not present in the tracker file(files_info.json) then the file that 
    has been requested to download didn't get to download previously (newly added file) 
    
    """
    if trackedFileIndex == -1:
        return False

    """ File name matched but upload_date doesn't then this is not pointing to the same file """
    if tracked_files[trackedFileIndex][1] != uploadDate:
        return False

    """ file present but there is no log for that file in files_info.json -> this shouldn't happen in any situation """

    """
    All files has been checked once then this shouldn't be checked anymore
    because there can be multiple files with the same name(in the website) and they all should be downloaded
    remove the found file so that the file with same name can be downloaded later on

    """
    tracked_files.pop(trackedFileIndex)
    downloaded_files.pop(downloadedFileIndex)

    return True


def downloadFile(cssSelector, files_count):
    download_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, cssSelector))
    )

    driver.execute_script(
        "arguments[0].target='_self';", download_button
    )  # to prevent it from opening into a new tab

    waitNSeconds(0.5)

    driver.execute_script("arguments[0].click();", download_button)

    return files_count + 1  # Keeping track of the downloaded files


###################################################################
# scroll down to load more files                                  #
# wait 60s before determining that there is no more files to load #
###################################################################
def loadMoreFiles(files_to_load, identifier, timeout=60, n_scroll=1):
    for _ in range(n_scroll):  # do this operation(scroll to load) for n times
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time_elapsed = 0
        current_len = len(files_to_load[0])

        ########################################################################
        # If the first one is loaded then all others will surely get loaded    #
        # so you don't have to check the whole length of 'files_to_load' array #
        # but if you want you can do something like this ->                    #
        # current_len = sum(len(i) for i in files_to_load)                     #
        ########################################################################
        while (current_len >= len(files_to_load[0])) and time_elapsed < timeout:
            waitNSeconds(1)

            # iterate through all the files that are needed to be loaded
            for idx, _ in enumerate(files_to_load):
                files_to_load[idx].extend(
                    [
                        element
                        for element in driver.find_elements(By.XPATH, identifier[idx])
                        if element not in files_to_load[idx]
                    ]
                )

            time_elapsed += 1

    return files_to_load


def waitToFinishDownload(directory, nfiles=None, timeout=network_failure_timeout):
    """
    Wait for downloads to finish with a specified timeout.

    Args
    ----
    directory : str
        The path to the folder where the files will be downloaded.
    timeout : int
        How many time_elapsed until it stops waiting.
    nfiles : int, defaults to None
        If provided, also wait for the expected number of files.

    """

    updateLog("\nWaiting for files to be downloaded")

    time_elapsed = 0
    dl_wait = True

    while dl_wait and time_elapsed < timeout:
        waitNSeconds(0.5)  # check every 0.5s
        dl_wait = False
        files = os.listdir(directory)

        # if nfiles and len(files) != nfiles:
        #     # if nfiles and len(files) < nfiles:
        #     dl_wait = True

        if not dl_wait:
            for fname in files:
                # as partial downloaded files will be of ".crdownload" extension for chromium based browsers
                if fname.endswith(".crdownload"):
                    dl_wait = True
                    break

        time_elapsed += 0.5

    if time_elapsed >= timeout:
        time_elapsed = -1
        updateLog(
            "\nYour connection is too slow or you are not connected! Try again later. Closing the connection..."
        )
    else:
        updateLog("\nContinuing after {}s...\n".format(time_elapsed))

    return time_elapsed


In [5]:
# waitNSeconds(5) # enable this if this script is fully automated for your case
driver.get("https://www.facebook.com/groups/201623576939858/files/")

###########################################################
# NOT NEEDED -> as it doesn't sort by file type reliably 😑
###########################################################
# sortButtons_xpath = "//div[@class='l9j0dhe7 du4w35lb j83agx80 pfnyh3mw taijpn5t bp9cbjyn owycx6da btwxx1t3 kt9q3ron ak7q8e6j isp2s0ed ri5dt5u2 rt8b4zig n8ej3o3l agehan2d sk4xxmp2 rq0escxv d1544ag0 tw6a2znq tdjehn4e tv7at329']" # find 3 per-scroll

# sortButtons = wait.until(
#     EC.presence_of_all_elements_located((By.XPATH, sortButtons_xpath))
# )

# # sort by file_type (PDF at the top in my case)
# driver.execute_script("arguments[0].click();", sortButtons[1])


In [8]:
j, k = 0, 1

# Target identifiers
# They change too often so they have to be present here
fileName_xpath = "//span[@class='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v lrazzd5p oo9gr5id hzawbc8m']"  # find 15 per-scroll
fileTypeDate_xpath = "//span[@class='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d9wwppkn iv3no6db e9vueds3 j5wam9gi b1v8xokw oo9gr5id hzawbc8m']"  # find 30 per-scroll

# Refresh existing files info every time this cell loads
[downloaded_files, tracked_files] = getExistingFilesInfo()

# Total files registered in the tracker file(files_info.json)
if tracked_files:
    with open(tracker_file_location, "r", encoding=encoding) as f:
        try:
            # Last registered file id
            registered_files_count = int(json.load(f)["files"][-1]["id"])
        except JSONDecodeError:
            registered_files_count = 0

####################################
# Grabbing initial loadeded target #
####################################
file_option_buttons = wait.until(
    EC.presence_of_all_elements_located((By.XPATH, fileOption_xpath))
)
file_names = wait.until(EC.presence_of_all_elements_located((By.XPATH, fileName_xpath)))
file_types_and_dates = wait.until(
    EC.presence_of_all_elements_located((By.XPATH, fileTypeDate_xpath))
)


for idx, button in enumerate(file_option_buttons):
    try:
        _name = file_names[idx].text
        _type = file_types_and_dates[j].text
        _date = file_types_and_dates[k].text

        # as 'file_types_and_dates' contains both file-type and date
        j += 2
        k += 2

        # Scrolling after every 15th check to load more files (as by default 15 files loads per scroll)
        if (idx + 1) % 15 == 0:
            [file_option_buttons, file_names, file_types_and_dates] = loadMoreFiles(
                [file_option_buttons, file_names, file_types_and_dates],
                [fileOption_xpath, fileName_xpath, fileTypeDate_xpath],
            )

            updateLog("\nTotal Loaded Files: {}\n".format(len(file_option_buttons)))

        ############################################################################
        # Download pdf files only                                                  #
        # Cannot check this before the scroll because                              #
        # there is a possibility to have more than one page worth of non-pdf files #
        ############################################################################
        if not _type == target_file_type.upper():
            updateLog(
                '\n😪 Skipping: "{}", 🤔 Reason: FILE_TYPE: "{}"\n'.format(_name, _type)
            )
            continue

        #################################
        # Skip already downloaded files #
        #################################
        if isDownloaded(_name, _date, downloaded_files, tracked_files):
            updateLog('😪 Skipping: "{}", 🤔 Reason: ALREADY_DOWNLOADED'.format(_name))
            continue

        #############################################################
        # If everything is OK then initiating file download         #
        #############################################################
        # this was giving me -> ElementClickInterceptedException -> #
        # wait.until(EC.element_to_be_clickable(button)).click()    #
        #############################################################

        driver.execute_script(
            "arguments[0].click();", button
        )  # Clicking the download button

        waitNSeconds(0.8)

        try:  # try to download the file
            registered_files_count = downloadFile(
                download_button_cssSelector, registered_files_count
            )

        except:
            updateLog(
                "\n*** Warning: No download link found at {} ***\n".format(idx + 1)
            )

        #################################################################
        # waiting at least 2s for safety sake 😅                        #
        # waiting (1.2+0.8+calculation_time) -> 2s+ after each download #
        #################################################################
        waitNSeconds(1.2)

        ##########################################################################################################
        # Waiting after every 'download_thread_count' downloads request                                          #
        # we have to wait for the files to be downloaded, if we continue without it there maybe some files       #
        # that couldn't be downloaded on time (before the link expire) and later on those files can't be resumed #
        ##########################################################################################################
        if (idx + 1) % download_thread_count == 0:
            download_time = waitToFinishDownload(
                secrets[2], registered_files_count
            )  # waiting for 600s before closing the connection for slow/no internet

            if download_time == -1:
                break  # Stoping the Bot because of a network failure

        # Updating info after a successful file download 😎
        appendFilesInfo(
            {
                "id": registered_files_count,
                "type": _type,
                "name": _name,
                "uploaded_date": _date,
            }
        )
        updateLog('✔ DONE: "{}", "{}", "{}"'.format(_type, _date, _name))

        break

    except Exception as e:
        updateLog("\n*** ERROR at {}, date: {} ---> {} ***\n".format(_name, _date, e))

# driver.close() # not closing because I had to run this cell multiple times during test
updateLog(
    "\n👏👏👏 Completed 👏👏👏\n\nFiles downloaded: {}\n".format(registered_files_count)
)
updateLog(
    "Go ahead and give this https://github.com/ShahriarDhruvo/Bots a star, thanks\t-- SED"
)


Skipping: "ভগিনী নিবেদিতা সেবিকা, বান্ধবী, মাতা.pdf", Reason: ALREADY_DOWNLOADED
Skipping: "বীজগণিতের সূচনায় ভারতীয় মনীষীদের অবদান – অশোককুমার সিংহ.pdf", Reason: ALREADY_DOWNLOADED
Skipping: "পাথর - হুমায়ুন আহমেদ.pdf", Reason: ALREADY_DOWNLOADED
Skipping: "হনিড লার্ক - অমিয়ভূষণ মজুমদার.pdf", Reason: ALREADY_DOWNLOADED
Skipping: "সান্যালদের কাহিনী- অমিয়ভূষণ মজুমদার.pdf", Reason: ALREADY_DOWNLOADED
Skipping: "একজন ক্রীতদাস - হুমায়ুন আহমেদ.pdf", Reason: ALREADY_DOWNLOADED
Skipping: "অতিথি - হুমায়ুন আহমেদ.pdf", Reason: ALREADY_DOWNLOADED
Skipping: "অচিন বৃক্ষ - হুমায়ুন আহমেদ.pdf", Reason: ALREADY_DOWNLOADED
Skipping: "Bangladesher Sahitye Nari.pdf", Reason: ALREADY_DOWNLOADED
Skipping: "RADHA.pdf", Reason: ALREADY_DOWNLOADED
Skipping: "সাদা মাকড়সা - অমিয়ভূষণ মজুমদার.pdf", Reason: ALREADY_DOWNLOADED
Skipping: "রসময়ীর রসিকতা - প্রভাতকুমার মুখোপাধ্যায়.pdf", Reason: ALREADY_DOWNLOADED
Skipping: "মধুরার ফ্ল্যাট এবং মিউজিয়াম (নাটক) - অমিয়ভূষণ মজুমদার.pdf", Reason: ALREADY_DOWNLOADED
Skipping